In [1]:
import sys
# !{sys.executable} -m pip install opencv-python
from IPython.core.debugger import set_trace
import cv2 
import numpy as np
from scipy import signal
from IPython.core.debugger import set_trace
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

In [2]:
def find_dft(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    edged = cv2.Canny(gray, 30, 200) 
    contours, hierarchy = cv2.findContours(edged,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    if len(contours)!=1:
        return [],[],False
    contours1 = np.reshape(contours,[-1,2])
    contours1 = np.divide(contours1,np.repeat(np.reshape(np.add(np.power(contours1[:,0],2),np.power(contours1[:,1],2)),[-1,1]),2,axis = 1))

    u_hat = contours1[:,0]+1j*contours1[:,1]
    u_hat[~np.isnan(u_hat).any(axis=0)]
    template = np.fft.fft(u_hat)
#     template_unit = template/np.absolute(template)

    return template,contours,True

image = cv2.imread('./sample.png') 
targetImage = cv2.imread('./a3.png') 

sheight,swidth = image.shape[0],image.shape[1]
height,width = targetImage.shape[0],targetImage.shape[1]

cv2.waitKey(0) 
  
# Grayscale 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
# Find Canny edges 
edged = cv2.Canny(gray, 30, 200) 
cv2.waitKey(0) 
  
# Finding Contours 
# Use a copy of the image e.g. edged.copy() 
# since findContours alters the image 
contours, hierarchy = cv2.findContours(edged,  
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 

template,_,_ = find_dft(image)
template = template[:50]
# template_abs = np.absolute(template)
# index = sorted(np.argsort(template_abs)[:50])
# template  = template[index]

# template_unit = np.divide(template,np.mean(np.absolute(template)))[:10]
temp_length = len(template)
# cv2.imshow('Canny Edges After Contouring', edged) 
# cv2.waitKey(0) 

print("Number of Contours found = " + str(len(contours))) 
  
# Draw all contours 
# -1 signifies drawing all contours 
cv2.drawContours(image, contours, -1, (0, 255, 0), 1) 
  
# cv2.imshow('Contours', image) 
# cv2.waitKey(0) 
cv2.destroyAllWindows() 



Number of Contours found = 1


In [3]:
# for h in range(sheight,height,5):
#     for w in range(swidth,width,5):
#         crop = targetImage[h-sheight:h,w-swidth:w]
#         stemplate,contour,checker = find_dft(crop)
# #         cv2.imshow('crops', crop) 
# #         cv2.waitKey(0) 
# #         cv2.destroyAllWindows() 
#         if checker and len(stemplate)>=temp_length and len(stemplate)>=80:
#             stemplate_abs = np.absolute(stemplate)
#             index1 = sorted(np.argsort(stemplate_abs)[:80])
#             stemplate_unit = np.divide(stemplate,np.absolute(stemplate))
#             stemplate_unit  = stemplate_unit[index1]
# #             dist = np.mean(np.power(np.absolute(template_unit - stemplate_unit),2))
#             dist = np.mean(np.power(np.imag(template_unit)-np.imag(stemplate_unit),2)+np.power(np.real(template_unit)-np.real(stemplate_unit),2))
# #             set_trace()
#             if dist:
#                 cv2.drawContours(crop, contour, -1, (0, 255, 0), 1) 
#                 cv2.imshow('crops', crop) 
#                 cv2.waitKey(0) 
#                 cv2.destroyAllWindows() 
#                 print(dist)
#         else:
#             continue
   

In [4]:
x = []
y = []

for h in range(sheight,height,2):
    for w in range(swidth,width,2):
        crop = targetImage[h-sheight:h,w-swidth:w]
        stemplate,contour,checker = find_dft(crop)
        
#         if checker and len(stemplate)>=temp_length and len(stemplate)>=80:
        if checker and len(contour)>0 and len(stemplate)>=50:
            stemplate = stemplate[:50]
#             dist1 = np.mean(np.power(np.imag(template)-np.imag(stemplate),2)+np.power(np.real(template)-np.real(stemplate),2))
            dist2 = np.mean(np.abs(np.absolute(template) - np.absolute(stemplate)))
            dist = dist2*100
            if dist<1.25 and not np.isnan(dist):
#                 cv2.rectangle(targetImage, ( w-swidth,h-sheight), (w,h), (255,0,0), 2)
#                 cv2.imshow('crop', crop) 
#                 cv2.waitKey(0) 
#                 cv2.destroyAllWindows() 
                x.append(h)
                y.append(w)
                print(dist)

for i,j in zip(x,y):
    cv2.rectangle(targetImage, ( j-swidth,i-sheight), (j,i), (255,0,0), 2)

cv2.imwrite("./output.jpg",targetImage)


0.902140786434002
0.529786938060527
0.18049476732351805
0.32555682916125966
0.7824108072171189
0.9741525327171177
0.5906079997225689
0.17971979953376296
0.2644754453024969
0.7349801894445751
1.243418923499682
1.0734492793649646
0.6856264177200968
0.26851005738086864
0.18336818047745432
0.6687794764270829
1.191771247227041
1.197740803897256
0.809811999192394
0.3940243902540272
0.13501733652752712
0.5606542704032229
1.0957542724181826
0.9585698635110665
0.5462825562590072
0.14338820799412544
0.44236908927612506
0.9518078070718295


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


1.2236689616228942
1.2012802788579002
1.2038778653377518
1.2362402979166267
1.2326693349032731
1.2183076142720646
1.2001464679969607
1.1807059326262177
1.2074329258116907
1.2440122959177973
1.2382449283625476
1.2194851816375192
1.2404919110276498
1.1957114382529392
1.1349068020436528
1.199196600774293
1.0385596413701583
1.0482310070984773
1.006359523338425
1.182837164405399
1.1471447995450845
1.0701010304236054
1.1752229851997438
1.1053104087718275
1.0508408221049983
1.0009058694930901
1.1946293759168622
1.1442808140215426
1.0915695112369053
1.0368860457736733
1.0794271300970244
1.2499837077255973
1.2092202687754763
1.1700675076235099
1.2248303410024817
1.2305318491120694
1.246244516711566
1.1631033579236494
1.0739647592900148
1.2227905069482967
1.1745075508772203
1.1301902643961326
1.2038306917230268
1.2266061650615847


True

NameError: name 'a' is not defined